## Imports

In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

In [2]:
tqdm.pandas()

In [3]:
import sys
sys.path.append('../src/')
from models.classification_methods import process_classification 

## definitions

In [4]:
model_name = 'neuralmind/bert-base-portuguese-cased'

In [5]:
random_seed = 42

In [6]:
path_raw_data = '../data/raw/'
path_processed_data = '../data/processed/'
path_results_cr = '../reports/classification_report'

In [7]:
list_target = ['ig','bo', 'cl', 'co', 'gl', 'lu']

## Classification

### Top mentioned timelines

In [8]:
# create a list of tuples with (data_train, data_test, target)

list_tuples_top_ment = []

for target in tqdm(list_target):
    
    path_data_train = path_raw_data + f'train_r3_{target}_top_mentioned_timelines.csv'
    path_data_test = path_raw_data + f'test_r3_{target}_top_mentioned_timelines.csv'

    data_train = pd.read_csv(
        path_data_train, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    data_test = pd.read_csv(
        path_data_test, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    
    list_tuples_top_ment.append((data_train, data_test, target))

 50%|█████     | 3/6 [00:10<00:10,  3.59s/it]

In [ ]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_top_ment
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')

### Users

In [ ]:
# create a list of tuples with (data_train, data_test, target)

list_tuples_users = []

for target in tqdm(list_target):
    
    path_data_train = path_raw_data + f'r3_{target}_train_users.csv'
    path_data_test = path_raw_data + f'r3_{target}_train_users.csv'

    data_train = pd.read_csv(
        path_data_train, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    data_test = pd.read_csv(
        path_data_test, 
        sep = ';', 
        encoding='utf-8-sig'
        )
    
    list_tuples_users.append((data_train, data_test, target))

#### Timelines

In [ ]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_users,
        X_cols=['Timeline']
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')

#### Stance

In [ ]:
# get results

classifier = DummyClassifier()

df_cr, df_test_results = process_classification(
        estimator = classifier,
        data_tuples = list_tuples_users,
        X_cols=['Stance']
)

df_cr[df_cr['class'] == 'macro avg'].sort_values('f1-score')